In [1]:
from basketball_reference_web_scraper import client
import json
import pandas as pd

In [2]:
competition_year = []
player_id = []
name = []
games_played = []
minutes_played = []

for year in range(2000, 2020):
    stats = client.players_season_totals(season_end_year=year)
    for result in stats:
        competition_year.append(year)
        player_id.append(result['slug'])
        name.append(result['name'])
        games_played.append(result['games_played'])
        minutes_played.append(result['minutes_played'])

In [3]:
player_id_2 = []
box_plus_minus = []

for year in range(2000, 2020):
    stats = client.players_advanced_season_totals(season_end_year=year)
    for result in stats:
        player_id_2.append(result['slug'])
        box_plus_minus.append(result['box_plus_minus'])

In [4]:
columns = ['year', 'player_id', 'name', 'games_played', 'minutes_played', 'player_id_2', 'box_plus_minus']

complete_nba_stats = pd.DataFrame(columns = columns)

In [5]:
complete_nba_stats['year'] = competition_year
complete_nba_stats['player_id'] = player_id
complete_nba_stats['name'] = name
complete_nba_stats['games_played'] = games_played
complete_nba_stats['minutes_played'] = minutes_played
complete_nba_stats['player_id_2'] = player_id_2
complete_nba_stats['box_plus_minus'] = box_plus_minus

In [6]:
# flatten season x player combinations & clean up df
complete_nba_stats.drop(['player_id_2'], axis = 1, inplace = True)

complete_nba_stats['wBPM'] = complete_nba_stats['minutes_played'] * complete_nba_stats['box_plus_minus']

complete_nba_stats = complete_nba_stats.groupby(['year', 'player_id', 'name']).sum()

complete_nba_stats['box_plus_minus'] = complete_nba_stats['wBPM'] / complete_nba_stats['minutes_played']

complete_nba_stats['minutes_per_game'] = complete_nba_stats['minutes_played'] / complete_nba_stats['games_played']

complete_nba_stats.drop(['wBPM'], axis = 1, inplace = True)

complete_nba_stats.reset_index(inplace = True)

complete_nba_stats = complete_nba_stats[['year', 'player_id', 'name', 'games_played', 
                                         'minutes_played', 'minutes_per_game', 'box_plus_minus']]

complete_nba_stats.head()

,year,player_id,name,games_played,minutes_played,minutes_per_game,box_plus_minus
0,2000,abdulta01,Tariq Abdul-Wahad,61,1578,25.868852,-1.201838
1,2000,abdursh01,Shareef Abdur-Rahim,82,3223,39.304878,2.200000
2,2000,alexaco01,Cory Alexander,29,329,11.344828,-2.400000
3,2000,allenra02,Ray Allen,82,3070,37.439024,3.600000
4,2000,alstora01,Rafer Alston,27,361,13.370370,-7.300000


In [7]:
complete_nba_stats.to_csv("../data/nba_stats.csv", index=False)